# DunedinPACE

## Index
1. [Define required clock parameters](#Define-required-clock-parameters)
2. [Download necessary data](#Download-necessary-data)
3. [Load data](#-Load-data)
4. [Extract features and weights](#Extract-features-and-weights)
5. [Load weights into pyaging model](#Load-weights-into-pyaging-model)
6. [Add reference values](#Add-reference-values)
7. [Add preprocessing and postprocesssing steps](#Add-preprocessing-and-postprocesssing-steps)
8. [Check all data objects](#Check-all-data-objects)
9. [Write clock dictionary](#Write-clock-dictionary)
10. [Clear directory](#Clear-directory)

Let's first import some packages:

In [1]:
import os
import marshal
import shutil
import json
import torch
import pandas as pd
import pyaging as pya

## Define required clock parameters

Let's define some required information first:

In [2]:
clock_name = 'dunedinpace'
data_type = 'methylation'
model_class = 'LinearModel'
species = 'Homo sapiens'
year = 2022
approved_by_author = '⌛'
citation = "Belsky, Daniel W., et al. \"DunedinPACE, a DNA methylation biomarker of the pace of aging.\" Elife 11 (2022): e73420."
doi = "https://doi.org/10.7554/eLife.73420"
notes = "The automatic failure if fewer than 80% of the CpG probes are available is not implemented and left to the user's discretion."

## Download necessary data

#### Download from R package

Let's first write an R script:

In [3]:
%%writefile download.r

options(repos = c(CRAN = "https://cloud.r-project.org/"))
library(jsonlite)

install.packages("devtools")
devtools::install_github("danbelsky/DunedinPACE", build_vignettes = FALSE)
library(DunedinPACE)

PACE_list = list(c(
    mPACE_Models$model_names, 
    mPACE_Models$gold_standard_probes, 
    mPACE_Models$model_weights, 
    mPACE_Models$model_intercept,
    mPACE_Models$model_means,
    mPACE_Models$model_probes,
    mPACE_Models$gold_standard_means    
))

write_json(PACE_list, "DunedinPACE.json", digits = 10)

Writing download.r


Let's run the R script. For this, R needs to be installed.

In [4]:
os.system("Rscript download.r")

0

## Load data

#### From JSON file

In [5]:
with open('DunedinPACE.json', 'r') as f:
    PACE_list = json.load(f)[0]

## Extract features and weights

First, let's extract the features and weights:

In [6]:
PACE_list.keys()

dict_keys(['1', 'DunedinPACE', 'DunedinPACE.1', 'DunedinPACE.2', 'DunedinPACE.3', 'DunedinPACE.4', 'DunedinPACE.5'])

Then, let's create lists for features and weights. Be careful about the intercept, as it usually shows up as a feature name.

In [7]:
features = PACE_list['DunedinPACE.4']
weights = torch.tensor(PACE_list['DunedinPACE.1']).unsqueeze(0).float()
intercept = torch.tensor([PACE_list['DunedinPACE.2'][0]]).float()

## Load weights into pyaging model

#### Linear model

In [8]:
model = pya.models.LinearModel(input_dim=len(features))

model.linear.weight.data = weights
model.linear.bias.data = intercept

model

LinearModel(
  (linear): Linear(in_features=173, out_features=1, bias=True)
)

## Add reference values

Some clocks have reference values in the case of missing features. It is also possible that these values are for preprocessing features rather than the clock features. Let's add a dictionary with the feature names as the keys.

In [9]:
reference_feature_values = dict(zip(PACE_list['DunedinPACE'], PACE_list['DunedinPACE.5']))

## Add preprocessing and postprocesssing steps

The preprocessing and postprocessing objects are dictionaries with the following format, with all items required. It takes in x in the form of a numpy array.

In [10]:
def preprocessing_function(x):
    """
    Apply quantile normalization on x using gold standard means.
    """
    gold_standard_means = preprocessing_helper_objects[0]

    # Sort the gold standard means
    sorted_gold_standard = np.sort(gold_standard_means)

    # Iterate through each row in x
    for i in range(x.shape[0]):
        # Sort the row data and store the original indices
        sorted_indices = np.argsort(x[i, :])
        sorted_data = x[i, sorted_indices]

        # Map the sorted data to their quantile values in the gold standard
        quantile_indices = np.round(
            np.linspace(0, len(sorted_gold_standard) - 1, len(sorted_data))
        ).astype(int)
        normalized_data = sorted_gold_standard[quantile_indices]

        # Re-order the normalized data to the original order
        original_order_indices = np.argsort(sorted_indices)
        x[i, :] = normalized_data[original_order_indices]

    return x
    
preprocessing_function_string = marshal.dumps(preprocessing_function.__code__)

preprocessing_helper_objects = [PACE_list['DunedinPACE.5']]

preprocessing = {
    'name': 'quantile_normalization_with_gold_standard',
    'preprocessing_function': preprocessing_function_string,
    'preprocessing_helper_objects': preprocessing_helper_objects
}

Similarly is the case of postprocessing. Remember that your function must be compatible with torch and is applied to each number individually.

In [11]:
postprocessing = None

## Check all data objects

Let's print all data objects to check if they make sense.

#### features

In [12]:
def my_print_function():
    print(f"There are {len(features)} features.")
    print(features)
pya.utils.print_to_scrollable_output(my_print_function)

#### reference_feature_values

In [13]:
def my_print_function():
    if reference_feature_values:
        print(f"There are {len(reference_feature_values)} reference feature values.")
    print(reference_feature_values)
pya.utils.print_to_scrollable_output(my_print_function)

#### preprocessing

In [14]:
def my_print_function():
    print(preprocessing)
    if preprocessing:
        print(preprocessing_helper_objects)
pya.utils.print_to_scrollable_output(my_print_function)

#### postprocessing

In [15]:
def my_print_function():
    print(postprocessing)
    if postprocessing:
        print(postprocessing)
pya.utils.print_to_scrollable_output(my_print_function)

#### weight_dict

In [16]:
def my_print_function():
    for name, param in model.named_parameters():
        print(f"Layer: {name}")
        print(f"Shape: {param.shape}")
        print(param.data)
pya.utils.print_to_scrollable_output(my_print_function)

## Write clock dictionary

Let's put everything together and save:

In [17]:
clock_dict = {
    # Metadata
    'clock_name': clock_name,
    'data_type': data_type,
    'model_class': model_class,
    'species': species,
    'year': year,
    'approved_by_author': approved_by_author,
    'citation': citation,
    'doi': doi,
    "notes": notes,

    # Data
    'reference_feature_values': reference_feature_values if reference_feature_values else None,
    'preprocessing': preprocessing if preprocessing else None, 
    'features': features,
    'weight_dict': model.state_dict(),
    'postprocessing': postprocessing if postprocessing else None,
}

torch.save(clock_dict, f'../weights/{clock_name}.pt')

## Clear directory

Delete all files that are not clock jupyter notebooks:

In [18]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted file: download.r
Deleted folder: .ipynb_checkpoints
Deleted file: DunedinPACE.json
